## Collecting Cap Data

This project will utilize cap data from Spotrac, as Spotrac is a comprehensive site that provides cap data for all NFL teams. Spotrac goes back to 2011, offering a decent range of historical data.

Select data from 2011 to 2024

In [5]:
# IMPORTS
import csv
import requests
import bs4
import pandas as pd

In [6]:
years = [x for x in range(2011, 2025, 1)]

In [7]:
teams = {
    'ARI': 'Arizona Cardinals',
    'ATL': 'Atlanta Falcons',
    'BAL': 'Baltimore Ravens',
    'BUF': 'Buffalo Bills',
    'CAR': 'Carolina Panthers',
    'CHI': 'Chicago Bears',
    'CIN': 'Cincinnati Bengals',
    'CLE': 'Cleveland Browns',
    'DAL': 'Dallas Cowboys',
    'DEN': 'Denver Broncos',
    'DET': 'Detroit Lions',
    'GB': 'Green Bay Packers',
    'HOU': 'Houston Texans',
    'IND': 'Indianapolis Colts',
    'JAX': 'Jacksonville Jaguars',
    'KC': 'Kansas City Chiefs',
    'LV': 'Las Vegas Raiders',
    'LAC': 'Los Angeles Chargers',
    'LAR': 'Los Angeles Rams',
    'MIA': 'Miami Dolphins',
    'MIN': 'Minnesota Vikings',
    'NE': 'New England Patriots',
    'NO': 'New Orleans Saints',
    'NYG': 'New York Giants',
    'NYJ': 'New York Jets',
    'PHI': 'Philadelphia Eagles',
    'PIT': 'Pittsburgh Steelers',
    'SF': 'San Francisco 49ers',
    'SEA': 'Seattle Seahawks',
    'TB': 'Tampa Bay Buccaneers',
    'TEN': 'Tennessee Titans',
    'WAS': 'Washington Commanders',
}

url_teams = {
    'ARI': 'arizona-cardinals',
    'ATL': 'atlanta-falcons',
    'BAL': 'baltimore-ravens',
    'BUF': 'buffalo-bills',
    'CAR': 'carolina-panthers',
    'CHI': 'chicago-bears',
    'CIN': 'cincinnati-bengals',
    'CLE': 'cleveland-browns',
    'DAL': 'dallas-cowboys',
    'DEN': 'denver-broncos',
    'DET': 'detroit-lions',
    'GB': 'green-bay-packers',
    'HOU': 'houston-texans',
    'IND': 'indianapolis-colts',
    'JAX': 'jacksonville-jaguars',
    'KC': 'kansas-city-chiefs',
    'LV': 'las-vegas-raiders',
    'LAC': 'los-angeles-chargers',
    'LAR': 'los-angeles-rams',
    'MIA': 'miami-dolphins',
    'MIN': 'minnesota-vikings',
    'NE': 'new-england-patriots',
    'NO': 'new-orleans-saints',
    'NYG': 'new-york-giants',
    'NYJ': 'new-york-jets',
    'PHI': 'philadelphia-eagles',
    'PIT': 'pittsburgh-steelers',
    'SF': 'san-francisco-49ers',
    'SEA': 'seattle-seahawks',
    'TB': 'tampa-bay-buccaneers',
    'TEN': 'tennessee-titans',
    'WAS': 'washington-commanders',
}

### Testing

In [8]:
# Testing
url = 'https://www.spotrac.com/nfl/philadelphia-eagles/overview/_/year/2024/sort/cap_total'

table_ind = {
    0: 'Active Roster',
    1: 'IR',
    2: 'Practice Squad',
    3: 'Dead Money'
}

text = requests.get(url).text

soup = bs4.BeautifulSoup(text, 'html.parser')
tables = soup.find_all('table')

In [9]:
def clean_name(word):
    split_pt = word.find('\n')
    return word[split_pt+1:]

def clean_header(word):
    split_pt = word.find('\n')
    return word[:split_pt]

### Active Roster

In [10]:
curr = tables[0]
headers = [header.text.strip() for header in curr.find_all('th')]
rows = []
for row in curr.find_all('tr')[1:]:
    rows.append([cell.text.strip() for cell in row.find_all('td')])

In [11]:
new_headers = [clean_header(x) if '\n' in x else x for x in headers]
new_headers[1] = 'Player'

In [12]:
active_df = pd.DataFrame(rows, columns=new_headers)
active_df.head()

,,Player,Pos,Age,Cap Hit,Cap Hit Pct,Dead Cap,Cash Total,Free Agent
0,1,Johnson\nLane Johnson,RT,34,"$15,865,000",6.21%,"($45,065,000)","$20,000,000",2027
1,2,Hurts\nJalen Hurts,QB,26,"$13,558,800",5.31%,"($120,871,200)","$40,000,000",2029
2,3,Brown\nA.J. Brown,WR,27,"$11,878,894",4.65%,"($73,220,682)","$21,250,000",2030
3,4,Mailata\nJordan Mailata,LT,27,"$11,666,000",4.57%,"($77,892,000)","$21,875,000",2029
4,5,Slay\nDarius Slay,CB,33,"$10,666,000",4.18%,"($33,357,932)","$11,350,000",2026


In [13]:
active_df.drop(columns=[''], inplace=True)

In [14]:
for col in ['Cap Hit', 'Dead Cap', 'Cash Total']:
    active_df[col] = active_df[col].str.replace('$', '').str.replace(',', '')
    if (col == 'Dead Cap'):
        active_df[col] = active_df[col].str.replace('(', '')
        active_df[col] = active_df[col].str.replace(')', '')
        active_df[col] = active_df[col].str.replace('-', '0')
    active_df[col] = active_df[col].astype(float)

In [15]:
active_df.head()

,Player,Pos,Age,Cap Hit,Cap Hit Pct,Dead Cap,Cash Total,Free Agent
0,Johnson\nLane Johnson,RT,34,15865000.0,6.21%,45065000.0,20000000.0,2027
1,Hurts\nJalen Hurts,QB,26,13558800.0,5.31%,120871200.0,40000000.0,2029
2,Brown\nA.J. Brown,WR,27,11878894.0,4.65%,73220682.0,21250000.0,2030
3,Mailata\nJordan Mailata,LT,27,11666000.0,4.57%,77892000.0,21875000.0,2029
4,Slay\nDarius Slay,CB,33,10666000.0,4.18%,33357932.0,11350000.0,2026


In [16]:
active_df['Cap Hit Pct'] = active_df['Cap Hit Pct'].str.replace('%', '').astype(float)
active_df['Player'] = active_df['Player'].apply(lambda x: clean_name(x))
active_df.head()

,Player,Pos,Age,Cap Hit,Cap Hit Pct,Dead Cap,Cash Total,Free Agent
0,Lane Johnson,RT,34,15865000.0,6.21,45065000.0,20000000.0,2027
1,Jalen Hurts,QB,26,13558800.0,5.31,120871200.0,40000000.0,2029
2,A.J. Brown,WR,27,11878894.0,4.65,73220682.0,21250000.0,2030
3,Jordan Mailata,LT,27,11666000.0,4.57,77892000.0,21875000.0,2029
4,Darius Slay,CB,33,10666000.0,4.18,33357932.0,11350000.0,2026


### IR

In [17]:
curr = tables[1]
headers = [header.text.strip() for header in curr.find_all('th')]
rows = []
for row in curr.find_all('tr')[1:]:
    rows.append([cell.text.strip() for cell in row.find_all('td')])

In [18]:
new_headers = [clean_header(x) if '\n' in x else x for x in headers]
new_headers[1] = 'Player'

In [19]:
ir_df = pd.DataFrame(rows, columns=new_headers)
ir_df.head()

,,Player,Pos,Age,Cap Hit,Cap Hit Pct,Dead Cap,Cash Total,Free Agent
0,1,Graham\nBrandon Graham,OLB,36,"$8,198,000",3.21%,"($17,216,000)","$4,000,000",2025
1,2,Bradberry\nJames Bradberry,S,31,"$4,305,000",1.69%,"($15,118,000)","$8,850,000",2026
2,3,Young\nByron Young,DT,24,"$1,007,669",0.39%,"($1,007,669)","$1,007,669",2027
3,4,Covey\nBritain Covey,WR,27,"$985,000",0.39%,-,"$985,000",2025
4,5,VanSumeren\nBen VanSumeren,ILB,24,"$915,000",0.36%,-,"$915,000",2025


In [20]:
for col in ['Cap Hit', 'Dead Cap', 'Cash Total']:
    ir_df[col] = ir_df[col].str.replace('$', '').str.replace(',', '')
    if (col == 'Dead Cap'):
        ir_df[col] = ir_df[col].str.replace('(', '')
        ir_df[col] = ir_df[col].str.replace(')', '')
        ir_df[col] = ir_df[col].str.replace('-', '0')
    ir_df[col] = ir_df[col].astype(float)

In [21]:
ir_df.drop(columns=[''], inplace=True)

In [22]:
ir_df['Cap Hit Pct'] = ir_df['Cap Hit Pct'].str.replace('%', '').astype(float)
ir_df['Player'] = ir_df['Player'].apply(lambda x: clean_name(x))

In [23]:
ir_df.head()

,Player,Pos,Age,Cap Hit,Cap Hit Pct,Dead Cap,Cash Total,Free Agent
0,Brandon Graham,OLB,36,8198000.0,3.21,17216000.0,4000000.0,2025
1,James Bradberry,S,31,4305000.0,1.69,15118000.0,8850000.0,2026
2,Byron Young,DT,24,1007669.0,0.39,1007669.0,1007669.0,2027
3,Britain Covey,WR,27,985000.0,0.39,0.0,985000.0,2025
4,Ben VanSumeren,ILB,24,915000.0,0.36,0.0,915000.0,2025


### Practice Squad

In [24]:
curr = tables[2]
headers = [header.text.strip() for header in curr.find_all('th')]
rows = []
for row in curr.find_all('tr')[1:]:
    rows.append([cell.text.strip() for cell in row.find_all('td')])

In [25]:
new_headers = [clean_header(x) if '\n' in x else x for x in headers]
new_headers[1] = 'Player'

In [26]:
ps_df = pd.DataFrame(rows, columns=new_headers)

In [27]:
ps_df

,,Player,Pos,Age,Cap Hit,Cap Hit Pct,Dead Cap,Cash Total
0,1,Toth\nBrett Toth,G,28,"$302,400",0.12%,-,"$302,400"
1,2,Gray\nDanny Gray,WR,25,"$285,600",0.11%,-,"$285,600"
2,3,Davis-Price\nTy Davis-Price,RB,24,"$225,000",0.09%,-,"$225,000"
3,4,Hall\nGabe Hall,DE,23,"$225,000",0.09%,-,"$225,000"
4,5,Sam\nAndre’ Sam,S,25,"$225,000",0.09%,-,"$225,000"
5,6,Vakalahi\nLaekin Vakalahi,T,21,"$225,000",0.09%,-,"$225,000"
6,7,Woods\nA.J. Woods,CB,23,"$212,500",0.08%,-,"$212,500"
7,8,Nickerson\nParry Nickerson,CB,30,"$201,600",0.08%,-,"$201,600"
8,9,Gant\nDallas Gant,ILB,24,"$112,500",0.04%,-,"$112,500"
9,10,Mathis\nOchaun Mathis,OLB,25,"$87,500",0.03%,-,"$87,500"


In [28]:
for col in ['Cap Hit', 'Dead Cap', 'Cash Total', 'Cap Hit Pct']:
    ps_df[col] = ps_df[col].str.replace('$', '').str.replace(',', '')
    ps_df[col] = ps_df[col].str.replace('(', '')
    ps_df[col] = ps_df[col].str.replace(')', '')
    ps_df[col] = ps_df[col].str.replace('-', '0')
    if (col == 'Cap Hit Pct'):
        ps_df[col] = ps_df[col].str.replace('%', '').astype(float)
    ps_df[col] = ps_df[col].astype(float)

In [29]:
ps_df.drop(columns=[''], inplace=True)

In [30]:
ps_df

,Player,Pos,Age,Cap Hit,Cap Hit Pct,Dead Cap,Cash Total
0,Toth\nBrett Toth,G,28,302400.0,0.12,0.0,302400.0
1,Gray\nDanny Gray,WR,25,285600.0,0.11,0.0,285600.0
2,Davis-Price\nTy Davis-Price,RB,24,225000.0,0.09,0.0,225000.0
3,Hall\nGabe Hall,DE,23,225000.0,0.09,0.0,225000.0
4,Sam\nAndre’ Sam,S,25,225000.0,0.09,0.0,225000.0
5,Vakalahi\nLaekin Vakalahi,T,21,225000.0,0.09,0.0,225000.0
6,Woods\nA.J. Woods,CB,23,212500.0,0.08,0.0,212500.0
7,Nickerson\nParry Nickerson,CB,30,201600.0,0.08,0.0,201600.0
8,Gant\nDallas Gant,ILB,24,112500.0,0.04,0.0,112500.0
9,Mathis\nOchaun Mathis,OLB,25,87500.0,0.03,0.0,87500.0


In [31]:
ps_df['Player'] = ps_df['Player'].apply(lambda x: clean_name(x))

In [32]:
ps_df.head()

,Player,Pos,Age,Cap Hit,Cap Hit Pct,Dead Cap,Cash Total
0,Brett Toth,G,28,302400.0,0.12,0.0,302400.0
1,Danny Gray,WR,25,285600.0,0.11,0.0,285600.0
2,Ty Davis-Price,RB,24,225000.0,0.09,0.0,225000.0
3,Gabe Hall,DE,23,225000.0,0.09,0.0,225000.0
4,Andre’ Sam,S,25,225000.0,0.09,0.0,225000.0


### Dead Cap

In [33]:
curr = tables[3]
headers = [header.text.strip() for header in curr.find_all('th')]
rows = []
for row in curr.find_all('tr')[1:]:
    rows.append([cell.text.strip() for cell in row.find_all('td')])

In [34]:
new_headers = [clean_header(x) if '\n' in x else x for x in headers]
new_headers[1] = 'Player'

In [35]:
new_headers

['', 'Player', 'Pos', 'Age', 'Cap Hit', 'Cap Hit Pct']

In [36]:
dead_df = pd.DataFrame(rows, columns=new_headers)

In [37]:
dead_df

,,Player,Pos,Age,Cap Hit,Cap Hit Pct
0,1,Reddick\nHaason Reddick,OLB,30,"$21,515,000",8.42%
1,2,Kelce\nJason Kelce,C,37,"$8,678,000",3.40%
2,3,Maddox\nAvonte Maddox,CB,28,"$7,719,532",3.02%
3,4,Cox\nFletcher Cox,DT,34,"$4,200,000",1.64%
4,5,Barnett\nDerek Barnett,DE,28,"$4,004,766",1.57%
...,...,...,...,...,...,...
81,82,Russell\nBrady Russell,TE,26,"$5,334",0.00%
82,83,Elliss\nNoah Elliss,DT,25,"$3,334",0.00%
83,84,Jobe\nJosh Jobe,CB,26,"$3,334",0.00%
84,85,Sills\nJosh Sills,G,26,"$3,334",0.00%


In [38]:
for col in ['Cap Hit', 'Cap Hit Pct']:
    dead_df[col] = dead_df[col].str.replace('$', '').str.replace(',', '')
    dead_df[col] = dead_df[col].str.replace('-', '')
    dead_df[col] = dead_df[col].str.replace('%', '')
    dead_df[col] = dead_df[col].astype(float)

In [39]:
dead_df.drop(columns=[''], inplace=True)

In [40]:
dead_df['Player'] = dead_df['Player'].apply(lambda x: clean_name(x))

In [45]:
active_df['status'] = 'Active'
ps_df['status'] = 'PS'
ir_df['status'] = 'IR'
dead_df['status'] = 'DeadCap'

In [46]:
combined_df = pd.concat([active_df, ir_df, ps_df, dead_df], ignore_index=True)
combined_df

,Player,Pos,Age,Cap Hit,Cap Hit Pct,Dead Cap,Cash Total,Free Agent,status
0,Lane Johnson,RT,34,15865000.0,6.21,45065000.0,20000000.0,2027,Active
1,Jalen Hurts,QB,26,13558800.0,5.31,120871200.0,40000000.0,2029,Active
2,A.J. Brown,WR,27,11878894.0,4.65,73220682.0,21250000.0,2030,Active
3,Jordan Mailata,LT,27,11666000.0,4.57,77892000.0,21875000.0,2029,Active
4,Darius Slay,CB,33,10666000.0,4.18,33357932.0,11350000.0,2026,Active
...,...,...,...,...,...,...,...,...,...
162,Brady Russell,TE,26,5334.0,0.00,NaN,NaN,NaN,DeadCap
163,Noah Elliss,DT,25,3334.0,0.00,NaN,NaN,NaN,DeadCap
164,Josh Jobe,CB,26,3334.0,0.00,NaN,NaN,NaN,DeadCap
165,Josh Sills,G,26,3334.0,0.00,NaN,NaN,NaN,DeadCap


In [47]:
combined_df.to_csv('eagles_2024.csv', index=False)

Link for SB winner each year: https://www.pro-football-reference.com/years/

Link for standings each year: https://www.pro-football-reference.com/years/2024/

## PFR Data

In [ ]:
url = 'https://www.pro-football-reference.com/super-bowl/'

text = requests.get(url).text

soup = bs4.BeautifulSoup(text, 'html.parser')
tables = soup.find_all('table')

In [ ]:
year = 2024
curr = tables[0]
rows = curr.find_all('tr')
clean_rows = []
for row in rows:
    clean_rows.append([cell.text.strip() for cell in row.find_all('td')])

sb_winners = [['Year', 'Winner', 'Loser']]

for row in clean_rows[1:]:
    if year <= 2010:
        break
    sb_winners.append([year, row[1], row[3]])
    year -= 1
    
sb_winners

### Standings

In [ ]:
url = 'https://www.pro-football-reference.com/years/2024/'

text = requests.get(url).text

soup = bs4.BeautifulSoup(text, 'html.parser')
afc_table = soup.find("table", {"id": "AFC"})
nfc_table = soup.find("table", {"id": "NFC"})

In [ ]:
# AFC
body = afc_table.find_all('tbody')
rows = body[0].find_all('tr')
rows
afc_standings = {}
for row in rows:
    if (type(row.find('a')) == type(None)):
        continue
    team_item = row.find_all('a')[0]
    team = team_item.text
    if team_item and team_item.next_sibling:
        playoff = team_item.next_sibling.strip()
    else:
        playoff = 'N'
    wins = row.find_all('td', {'data-stat': 'wins'})[0].text
    losses = row.find_all('td', {'data-stat': 'losses'})[0].text
    win_pct = row.find_all('td', {'data-stat': 'win_loss_perc'})[0].text
    afc_standings[team] = [wins, losses, win_pct, playoff]

In [ ]:
# AFC
body = nfc_table.find_all('tbody')
rows = body[0].find_all('tr')
rows
nfc_standings = {}
for row in rows:
    if (type(row.find('a')) == type(None)):
        continue
    team_item = row.find_all('a')[0]
    team = team_item.text
    if team_item and team_item.next_sibling:
        playoff = team_item.next_sibling.strip()
    else:
        playoff = 'N'
    wins = row.find_all('td', {'data-stat': 'wins'})[0].text
    losses = row.find_all('td', {'data-stat': 'losses'})[0].text
    win_pct = row.find_all('td', {'data-stat': 'win_loss_perc'})[0].text
    nfc_standings[team] = [wins, losses, win_pct, playoff]